https://earthquake.usgs.gov/earthquakes/search/

In [1]:
from datetime import datetime

import pandas as pd

from planet.app import Planet
from parser.repo.handler import QuakeExtractor

In [ ]:
start = datetime(2023,3,6)
end = datetime(2023,3,15)

extract = QuakeExtractor(start, end)
extract.period_list

In [4]:
extract.remote_extract()

quake_df = extract.to_frame()
quake_df.columns = ['Магнитуда', 'Долгота', 'Широта', 'Дата']

In [5]:
def add_param(frame):   
    planet = Planet(frame['Долгота'], frame['Широта'], frame['Дата'])

    return pd.concat([frame, pd.Series(planet.get_parametrs())])

In [6]:
quake_df = quake_df.apply(add_param, axis=1)

In [7]:
quake_df

,Магнитуда,Долгота,Широта,Дата,Mercury_earth_distance,Mercury_sun_distance,Mercury_hlat,Mercury_hlon,Mercury_size,Mercury_radius,...,Moon_a_ra,Moon_elong,Sun_earth_distance,Sun_sun_distance,Sun_hlat,Sun_hlon,Sun_size,Sun_radius,Sun_a_ra,Sun_elong
0,3.9,-169.8396,50.555,2023-03-15 05:35:25,203.974573,55.928647,-0.109002,6.032886,4.943213,0.000012,...,4.568880,-1.590822,148.730112,0.0,0.000003,3.039356,1930.458862,0.004680,6.184135,0.0
1,4.1,160.0092,51.313,2023-03-15 04:31:00,203.974841,55.928647,-0.109002,6.032886,4.943206,0.000012,...,4.568880,-1.590822,148.730433,0.0,0.000003,3.039356,1930.454712,0.004680,6.184135,0.0
2,2.5,-178.0788,51.8415,2023-03-15 03:35:57,203.974645,55.928647,-0.109002,6.032886,4.943211,0.000012,...,4.568880,-1.590822,148.730183,0.0,0.000003,3.039356,1930.457886,0.004680,6.184135,0.0
3,3.9,170.5401,54.3841,2023-03-15 02:37:54,203.974930,55.928647,-0.109002,6.032886,4.943203,0.000012,...,4.568880,-1.590822,148.730495,0.0,0.000003,3.039356,1930.453857,0.004680,6.184135,0.0
4,2.9,-178.2475,51.2748,2023-03-14 21:34:04,204.366321,56.794929,-0.112794,5.958735,4.933737,0.000012,...,4.307330,-1.811825,148.688783,0.0,0.000002,3.021950,1930.995483,0.004681,6.168131,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,2.9,179.1516,51.5751,2023-03-07 12:20:23,204.056358,62.454037,-0.121999,5.494206,4.941231,0.000012,...,2.846119,3.015539,148.410697,0.0,-0.000002,2.899884,1934.613647,0.004690,6.055506,0.0
213,5.2,144.5598,42.3233,2023-03-07 05:24:47,204.056108,62.454037,-0.121999,5.494206,4.941237,0.000012,...,2.846119,3.015539,148.410884,0.0,-0.000002,2.899884,1934.611206,0.004690,6.055506,0.0
214,4.5,141.764,42.3499,2023-03-07 04:07:27,204.056233,62.454037,-0.121999,5.494206,4.941234,0.000012,...,2.846119,3.015539,148.411027,0.0,-0.000002,2.899884,1934.609375,0.004690,6.055506,0.0
215,3.7,178.3193,51.9329,2023-03-06 14:17:41,203.642193,63.168428,-0.121301,5.434356,4.951281,0.000012,...,2.648003,2.834591,148.372988,0.0,-0.000002,2.882411,1935.105347,0.004691,6.039310,0.0
